<a href="https://colab.research.google.com/github/rlawogjs96/text_mining/blob/main/Code6_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sentiment Analysis

### Machine learning approach

### (한국어) Movies review data
- https://shinminyong.tistory.com/13
- https://wonhwa.tistory.com/35
- https://cyc1am3n.github.io/2018/11/10/classifying_korean_movie_review.html
- https://techblog-history-younghunjo1.tistory.com/111


In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 지도학습 (Machine learning) 기반
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, roc_auc_score
import re

In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 610 kB/s 
     |████████████████████████████████| 448 kB 76.4 MB/s 


#### 데이터 불러오기 및 전처리 과정

In [ ]:
with open('/content/drive/MyDrive/2022년도 겨울 특강-텍스트마이닝/Text_mining_4강/data/2016_movie_reviews.txt',encoding = 'utf-8') as f:
          docs = [doc.strip().split('\t\t') for doc in f]

In [ ]:
import konlpy.tag

okt = konlpy.tag.Okt()

In [ ]:
Features = ['Noun', 'Verb','Adjective']

def select_word(text):
    text = text.strip()
    okt_morphs = okt.pos(text)
    selected_words = []
    for word, pos in okt_morphs:
        if pos in Features:
            selected_words.append(word)
    
    return ' '.join(selected_words)

In [ ]:
with open('2016_filtered_review1.text','w', encoding='utf-8') as f1:
    for doc in docs:
        if len(doc) == 3:
            review = doc[1]
            doc_review = doc[0]+'\t\t'+select_word(review)+'\t\t'+doc[2]+'\n'
            f1.write(doc_review)

In [ ]:
with open('/content/drive/MyDrive/2022년도 겨울 특강-텍스트마이닝/Text_mining_4강/data/2016_filtered_review.txt', encoding = 'utf-8') as f:
    docs = [doc.strip().split('\t\t') for doc in f]
    docs = [(doc[1], int(doc[2])) for doc in docs if len(doc) == 3]
    # To read the second and third column info from each row
    texts, scores = zip(*docs)

In [ ]:
filtered_texts = []
filtered_labels = []

for text, score in zip(texts, scores):
    if 4 < score < 8:
        continue
        
    # 평점 기준으로 문서에 label을 부여
    # 1 ~ 4 -> 부정, 0
    # 8 ~ 10 -> 긍정, 1
    filtered_texts.append(text)
    filtered_labels.append(1 if score >= 8 else 0)

In [ ]:
len(filtered_texts)

672606

In [ ]:
# To split the data into training and test datasets
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(filtered_texts, filtered_labels, test_size=0.1, random_state=0)

In [ ]:
train_texts[1]

' 재밌고 감동 시간 긴 러닝 타임 정말 유쾌하 훅 지나갔'

#### 머신러닝 모델 학습

##### TF 정보 이용해서 표현하기

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
tf_vector = CountVectorizer(min_df=1, ngram_range=(1,1)) 
tf_train_features = tf_vector.fit_transform(train_texts) 
tf_test_features = tf_vector.transform(test_texts)

In [ ]:
vocablist = [word for word, _ in sorted(tf_vector.vocabulary_.items(), key=lambda x:x[1])]
print(len(vocablist))
vocablist[:10]

84278


['가가', '가가호호', '가각', '가감', '가거', '가겄', '가겄소', '가게', '가겠', '가격']

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# tf matrix를 사용한 경우
lr_tf = LogisticRegression(max_iter=100)
lr_tf.fit(tf_train_features, train_labels) # 학습
pred_labels = lr_tf.predict(tf_test_features) # 테스트

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
# 정확도 확인하기
from sklearn.metrics import accuracy_score
print('Misclassified samples: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
print('Accuracy: %.2f' % accuracy_score(test_labels, pred_labels))

Misclassified samples: 4869 out of 67261
Accuracy: 0.93


In [ ]:
review_index = 6
print(test_texts[review_index])
print("The predicted value: {} and probability: {}".format(lr_tf.predict(tf_test_features[review_index]), lr_tf.predict_proba(tf_test_features[review_index])))

 초반 분 노잼 미리 예습 마블 보고 가면 좀더 허니 잼 중후 반 누구 꿀잼 느끼 후반 또 감독 및 작가 어어 어 하면 어리 버리 타면 찍었 또 노잼 스크린 내려가고 또 나오니 다시 서 쪽 팔 시름 앉아 기다리 긔 추천
The predicted value: [1] and probability: [[0.252201 0.747799]]


In [ ]:
# Get coefficients of the model 
coefficients = lr_tf.coef_.tolist()

sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)

print(sorted_coefficients[:5])
# print top 50 positive words
for word, coef in sorted_coefficients[:50]:
    print('{0:} ({1:.3f})'.format(vocablist[word], coef))
# print top 50 negative words
for word, coef in sorted_coefficients[-50:]:
    print('{0:} ({1:.3f})'.format(vocablist[word], coef))

[(65454, 3.7160813247822073), (80550, 3.7062792526669295), (61942, 3.673928333283681), (10337, 3.595313222490544), (80936, 3.4160160210331902)]
존잼 (3.716)
핵꿀잼 (3.706)
재밌던 (3.674)
꿀잼 (3.595)
행복했 (3.416)
기대됩 (3.404)
재밋었어 (3.395)
재밋는데 (3.379)
영화평론가 (3.370)
걱정했 (3.263)
만족스러웠 (3.246)
재밌구 (3.202)
졸잼 (3.128)
재밌었 (3.067)
짱잼 (3.066)
개꿀잼 (3.036)
사랑스러운 (3.007)
군더더기 (3.006)
매혹 (2.985)
기대되는 (2.911)
재미있게 (2.899)
재밋엇어 (2.897)
십점 (2.867)
재밋네 (2.858)
흠잡 (2.857)
완벽 (2.815)
핵잼 (2.804)
잇게 (2.798)
꾸르잼 (2.794)
행복해 (2.791)
황홀 (2.781)
완벽했 (2.760)
저격 (2.736)
메이드 (2.709)
개잼 (2.663)
사랑해 (2.641)
지나갔 (2.621)
재밋음 (2.618)
만족한 (2.606)
매너 (2.588)
강추 (2.585)
재미있었 (2.564)
긴장하면 (2.554)
놓칠 (2.541)
재밌고 (2.530)
훌륭합 (2.516)
사랑합 (2.503)
기대되 (2.502)
악플 (2.494)
짱짱맨 (2.488)
건질 (-2.743)
차라리 (-2.744)
아까워 (-2.770)
가관 (-2.779)
용가리 (-2.783)
실망함 (-2.824)
혜선 (-2.825)
별루임 (-2.836)
망쳤 (-2.838)
페미 (-2.843)
낭비 (-2.851)
무표정 (-2.856)
수면제 (-2.858)
접함 (-2.872)
졸작 (-2.873)
망작 (-2.874)
엉망 (-2.876)
빵점 (-2.905)
심형래 (-2.915)
별루네 (-2.923)
더러워 (-2.93

##### TF-IDF 정보 이용해서 표현하기

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf_vector = TfidfVectorizer(min_df=1, ngram_range=(1,1)) 
tfidf_train_features = tfidf_vector.fit_transform(train_texts)
tfidf_test_features = tfidf_vector.transform(test_texts)

In [ ]:
lr_tfidf = LogisticRegression(max_iter = 100)
lr_tfidf.fit(tfidf_train_features, train_labels) #학습
pred_labels_tfidf = lr_tfidf.predict(tfidf_test_features)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
print('Misclassified samples: {} out of {}'.format((pred_labels_tfidf != test_labels).sum(),len(test_labels)))
print('Accuracy: %.2f' % accuracy_score(test_labels, pred_labels_tfidf))

Misclassified samples: 4804 out of 67261
Accuracy: 0.93


In [ ]:
# Get coefficients of the model 
coefficients = lr_tfidf.coef_.tolist()

sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)

print(sorted_coefficients[:5])
# print top 50 positive words
for word, coef in sorted_coefficients[:50]:
    print('{0:} ({1:.3f})'.format(vocablist[word], coef))
# print top 50 negative words
for word, coef in sorted_coefficients[-50:]:
    print('{0:} ({1:.3f})'.format(vocablist[word], coef))

[(61907, 9.009859903818958), (61983, 8.80962599358566), (10337, 8.03508593887329), (61152, 7.332592705186325), (61964, 6.947982893204875)]
재밌게 (9.010)
재밌었 (8.810)
꿀잼 (8.035)
재미있게 (7.333)
재밌어 (6.948)
여운 (6.677)
재미있었 (6.606)
최고 (6.232)
강추 (5.921)
존잼 (5.835)
재밌고 (5.448)
가슴 (5.240)
재밌던 (5.230)
재밌 (5.131)
심장 (4.963)
대박 (4.959)
충분히 (4.900)
재밌네 (4.852)
수작 (4.814)
약간 (4.809)
개꿀잼 (4.763)
즐겁 (4.741)
재미있어 (4.732)
빠져 (4.731)
재미있고 (4.720)
다만 (4.673)
최고다 (4.638)
가는 (4.590)
감사 (4.548)
지루하지 (4.501)
슬펐 (4.455)
재밌는 (4.392)
재미있네 (4.389)
유쾌 (4.387)
흥미진진 (4.383)
졸잼 (4.379)
괜찮 (4.247)
감탄 (4.218)
재미있 (4.185)
살짝 (4.173)
만족 (4.161)
걱정했 (4.151)
압도 (4.144)
테러 (4.144)
울었 (4.139)
멋지 (4.123)
감사합 (4.097)
사랑해 (4.074)
호강 (4.069)
굿굿 (4.064)
이도 (-4.337)
낚였 (-4.352)
아까웠 (-4.367)
어색하고 (-4.411)
졸았 (-4.500)
짜증 (-4.511)
지루해서 (-4.559)
빠순이 (-4.583)
클레멘타인 (-4.608)
별로 (-4.616)
지루했 (-4.625)
퇴보 (-4.629)
속았 (-4.664)
만드냐 (-4.684)
숙면 (-4.705)
집기 (-4.724)
거품 (-4.728)
이하 (-4.750)
과대 (-4.802)
미화 (-4.847)
거르 (-4.847)
댓글알바 (-4.861)
역사왜곡 (

#### 딥러닝 기반 모델학습

In [ ]:
# 정수 인코딩
dictionary = []
for areview in filtered_texts:
    dict_review = areview.split()
    dictionary.append(dict_review)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(dictionary)

print(tokenizer.word_index)

{'영화': 1, '좋': 2, '봤': 3, '너무': 4, '연기': 5, '진짜': 6, '정말': 7, '보고': 8, '감동': 9, '했': 10, '볼': 11, '배우': 12, '스토리': 13, '였': 14, '더': 15, '보는': 16, '입니': 17, '잘': 18, '것': 19, '최고': 20, '본': 21, '생각': 22, '재미': 23, '재밌게': 24, '사람': 25, '시간': 26, '봐': 27, '아이': 28, '이': 29, '기대': 30, '재밌었': 31, '점': 32, '역시': 33, '잼': 34, '하는': 35, '있는': 36, '수': 37, '그냥': 38, '평점': 39, '좀': 40, '내용': 41, '꼭': 42, '알': 43, '아니': 44, '이런': 45, '마지막': 46, '좀비': 47, '그': 48, '재밌어': 49, '할': 50, '재미있게': 51, '말': 52, '넘': 53, '느낌': 54, '있었': 55, '때': 56, '나': 57, '장면': 58, '같아': 59, '모르': 60, '보면': 61, '없는': 62, '왜': 63, '같은': 64, '보기': 65, '정도': 66, '재미있었': 67, '완전': 68, '처음': 69, '액션': 70, '다시': 71, '내': 72, '감독': 73, '또': 74, '한': 75, '한번': 76, '돈': 77, '중간': 78, '들': 79, '눈물': 80, '하나': 81, '듯': 82, '왔': 83, '이었': 84, '짱': 85, '함': 86, '분': 87, '보세': 88, '재밌': 89, '마음': 90, '괜찮': 91, '거': 92, '합': 93, '안': 94, '우리': 95, '영상': 96, '끝': 97, '조금': 98, '마블': 99, '와': 100, '저': 101, '살': 102, '되': 103, '역사': 1

In [ ]:
threshold = 3
total_cnt = len(tokenizer.word_index)
rare_cnt = 0

total_freq = 0
rare_freq = 0

for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어집합의 크기 : ', total_cnt)
print('등장빈도가 {}번 이하인 희귀 단어의 수 : {}'.format(threshold-1, rare_cnt))
print('단어 집합에서 희귀 단어의 비율 : ', (rare_cnt/total_cnt)*100)
print('전체 등장 빈도에서 희귀 단어 등장 빈도 비율 : ', (rare_freq/total_freq)*100)

단어집합의 크기 :  91339
등장빈도가 2번 이하인 희귀 단어의 수 : 52365
단어 집합에서 희귀 단어의 비율 :  57.330384611173756
전체 등장 빈도에서 희귀 단어 등장 빈도 비율 :  0.9072362510191964


In [ ]:
vocab_size = total_cnt - rare_cnt +1
print('단어 집합의 크기 : ', vocab_size)

단어 집합의 크기 :  38975


In [ ]:
x_train = [list(anreview) for anreview in train_texts]
x_test = [list(anreview) for anreview in test_texts]

In [ ]:
tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(x_train)
x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

In [ ]:
print(x_train[:3])

[[1, 35, 62, 1, 168, 48, 4, 1, 4, 34, 1, 89, 101, 6, 1, 2, 3], [1, 5, 30, 10, 1, 27, 44, 1, 19, 51, 1, 152, 1, 104, 566, 1, 143, 140, 1, 25, 43, 1, 108, 330, 7, 1, 967, 1, 12, 15, 321], [1, 35, 62, 1, 130, 1, 18, 19, 1, 11, 10, 1, 55, 1, 235, 1, 77, 1, 130, 1, 17, 102, 180, 1, 131, 1, 52, 1, 40, 165, 1, 163, 77, 1, 140, 1, 130, 1, 181, 91, 1, 130, 1, 225, 91, 140]]


In [ ]:
# 샘플의 길이 맞추기
x_train = pad_sequences(x_train, maxlen = 30)
x_test = pad_sequences(x_test, maxlen = 30)

In [ ]:
y_train = np.array(train_labels)
y_test = np.array(test_labels)

##### LSTM으로 감성분류하기

In [ ]:
# LSTM
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
# 하이퍼파라미터인 임베딩 벡터의 차원은 100, 은닉 상태의 크기는 128
embedding_dim = 100
hidden_units = 128

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(x_train, y_train, epochs=15, callbacks=[es, mc], batch_size=64, validation_split=0.2)

Epoch 1/15
7567/7567 [==============================] - ETA: 0s - loss: 0.2440 - acc: 0.9033
Epoch 00001: val_acc improved from -inf to 0.91270, saving model to best_model.h5
7567/7567 [==============================] - 479s 63ms/step - loss: 0.2440 - acc: 0.9033 - val_loss: 0.2190 - val_acc: 0.9127
Epoch 2/15
7567/7567 [==============================] - ETA: 0s - loss: 0.2137 - acc: 0.9156
Epoch 00002: val_acc improved from 0.91270 to 0.91743, saving model to best_model.h5
7567/7567 [==============================] - 479s 63ms/step - loss: 0.2137 - acc: 0.9156 - val_loss: 0.2066 - val_acc: 0.9174
Epoch 3/15
7567/7567 [==============================] - ETA: 0s - loss: 0.2013 - acc: 0.9205
Epoch 00003: val_acc improved from 0.91743 to 0.91987, saving model to best_model.h5
7567/7567 [==============================] - 458s 60ms/step - loss: 0.2013 - acc: 0.9205 - val_loss: 0.2008 - val_acc: 0.9199
Epoch 4/15
7566/7567 [============================>.] - ETA: 0s - loss: 0.1925 - acc: 0.924

In [ ]:
# 임의의 리뷰를 예측하는 함수

def sentiment_predict(new_sentence):
  new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
  new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
  new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
  encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
  pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
  score = float(loaded_model.predict(pad_new)) # 예측
  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))

In [ ]:
sentiment_predict('이 영화 너무 재밌다')

In [ ]:
sentiment_predict('아 영화 너무 재미없네')

### (영어) Reviews of fine foods from amazon

- https://www.kaggle.com/laowingkin/amazon-fine-food-review-sentiment-analysis/data


*ProductId* : Unique identifier for the product

*UserId* : Unqiue identifier for the user

*ProfileName* : Profile name of the user

*HelpfulnessNumerator* : Number of users who found the review helpful

*HelpfulnessDenominator* : Number of users who indicated whether they found the review helpful or not

*Score* : Rating between 1 and 5

*Time* : Timestamp for the review

*Summary* : Brief summary of the review

*Text* : Text of the review

#### 데이터 불러오기 및 전처리과정

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
review_amazon = pd.read_csv('/content/drive/MyDrive/2022년도 겨울 특강-텍스트마이닝/Text_mining_4강/data/Reviews.csv')
review_amazon

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
568449,568450,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0,0,5,1299628800,Will not do without,Great for sesame chicken..this is a good if no...
568450,568451,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0,0,2,1331251200,disappointed,I'm disappointed with the flavor. The chocolat...
568451,568452,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2,2,5,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o..."
568452,568453,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1,1,5,1331596800,Favorite Training and reward treat,These are the BEST treats for training and rew...


In [ ]:
review_amazon.describe()

,Id,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time
count,568454.000000,568454.000000,568454.00000,568454.000000,5.684540e+05
mean,284227.500000,1.743817,2.22881,4.183199,1.296257e+09
std,164098.679298,7.636513,8.28974,1.310436,4.804331e+07
min,1.000000,0.000000,0.00000,1.000000,9.393408e+08
25%,142114.250000,0.000000,0.00000,4.000000,1.271290e+09
50%,284227.500000,0.000000,1.00000,5.000000,1.311120e+09
75%,426340.750000,2.000000,2.00000,5.000000,1.332720e+09
max,568454.000000,866.000000,923.00000,5.000000,1.351210e+09


In [ ]:
review_df = review_amazon[review_amazon['Score'] != 3]
texts_en = list(review_df['Text'])
labels_dict = {1:0, 2:0, 4:1, 5:1}
labels_en = list(review_df['Score'].map(labels_dict))

In [ ]:
print(len(result_texts))
print(len(labels_en))

525814
525814


In [ ]:
review_df.groupby('Score').size()

Score
1     52268
2     29769
4     80655
5    363122
dtype: int64

In [ ]:
# 영어 데이터 토큰화
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import re

# html 태그 문자 삭제하기
def remove_tag(text):
    tag_pattern = re.compile('<.*?>')
    clean_text = re.sub(tag_pattern, '', text)
    return clean_text

In [ ]:
result_texts = []
for antext in texts_en:
    antext = remove_tag(antext)
    anlist = nltk.word_tokenize(antext.lower())
    result_text = ' '.join(anlist)
    result_texts.append(result_text)

#### 분류 모델 적용 함수

In [ ]:
# 모델 적용 함수
def model_fit(x_text, y_label, model, clf_model, coef_show=1):
  # 학습 text, 정답 label, 사용 feature model, 분류 model, 테스트set 크기(1~0)

  # text feature 학습시키기
    x_c = model.fit_transform(x_text)
    print('# features: {}'.format(x_c.shape[1]))
  
  # train, test set으로 나누기
    x_train, x_test, y_train, y_test = train_test_split(x_c, y_label, test_size = 0.1, random_state=0)
    print('# 학습데이터 개수 : {}'.format(x_train.shape[0]))
    print('# 테스트데이터 개수 : {}'.format(x_test.shape[0]))

    clf = clf_model.fit(x_train, y_train)
    acc = clf.score(x_test, y_test)
    print('Model Accuracy: {}'.format(acc))

  # 모델 계수 나타내기
    if coef_show == 1:
        w = model.get_feature_names()
        coef = clf.coef_.tolist()[0]
        coef_df = pd.DataFrame({'Word' : w, 'Coefficient': coef})
        coef_df = coef_df.sort_values(['Coefficient', 'Word'], ascending = [0,1])
        
        print('')
        print('-Top 20 positive-')
        print(coef_df.head(20).to_string(index = False))
        print('')
        print('-Top 20 negative-')
        print(coef_df.tail(20).to_string(index=False))

    return clf_model

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

c = CountVectorizer(stop_words= 'english')
logistic_ct = LogisticRegression()
model_fit(result_texts, labels_en, c, logistic_ct)

# features: 128114
# 학습데이터 개수 : 473232
# 테스트데이터 개수 : 52582


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Model Accuracy: 0.937963561675098

-Top 20 positive-
       Word  Coefficient
 pleasantly     3.759825
  addicting     3.279950
   downside     3.110088
   drawback     2.824347
    easiest     2.676898
     resist     2.643892
     delish     2.461217
    worries     2.430361
    welcome     2.343111
  skeptical     2.314842
    trainer     2.293811
     intend     2.245071
       whim     2.215846
    resolve     2.160892
   tastiest     2.160451
        yay     2.146528
competitors     2.142110
    gobbles     2.130385
   soothing     2.106280
     bertie     2.105381

-Top 20 negative-
          Word  Coefficient
      mediocre    -2.654775
       allegro    -2.657742
disappointment    -2.664843
    unbearable    -2.669608
        lacked    -2.721153
    unfinished    -2.768892
     cancelled    -2.775537
         worst    -2.808503
 dissapointing    -2.816588
   unappealing    -2.838896
 disappointing    -2.858846
     redeeming    -2.907075
           ick    -2.964236
    returna

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

tfidf = TfidfVectorizer(stop_words = 'english')
logistic_tfidf = LogisticRegression()
tfidf_model = model_fit(result_texts, labels_en, tfidf, logistic_tfidf)

# features: 128114
# 학습데이터 개수 : 473232
# 테스트데이터 개수 : 52582


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Model Accuracy: 0.9359096268685101

-Top 20 positive-
      Word  Coefficient
     great    13.369479
 delicious    12.336478
      best    12.283729
   perfect    10.440572
     loves     9.780830
 excellent     9.597805
    highly     9.152440
    hooked     8.317255
      love     8.226835
   amazing     7.947705
pleasantly     7.942464
 wonderful     7.887874
   awesome     7.788876
  favorite     7.247112
      nice     7.229520
 fantastic     7.125268
      good     7.107797
     yummy     7.032855
    smooth     6.691618
   pleased     6.681935

-Top 20 negative-
          Word  Coefficient
       sounded    -6.191954
          yuck    -6.209747
         stale    -6.295390
         hopes    -6.297070
         worse    -6.306521
    disgusting    -6.382048
         bland    -6.479666
        return    -6.485945
     tasteless    -6.710220
   undrinkable    -6.818141
         threw    -6.860730
          weak    -6.949682
      horrible    -7.681028
 unfortunately    -7.810392
  d

## Lexicon based

In [ ]:
# Simple example

pos_words = ['happy', 'fun', 'great', 'exciting', 'fantastic']
neg_words = ['boring', 'disappointing', 'bad', 'unpleasant']

In [ ]:
import re

def get_sent(sentence):
    sentence = re.sub('[^\s\d\w]', '', sentence)
    words = sentence.lower().split()

    pos_score = 0
    neg_score = 0

    for word in words:
        if word in pos_words:
            pos_score +=1 
        elif word in neg_words:
            neg_score -=1
    total_score = pos_score + neg_score
    
    return total_score

In [ ]:
example1 = 'The move was fun.'

score = get_sent(example1)
print(score)

1


In [ ]:
example2 = 'This one is so boring and disappointing'

score = get_sent(example2)
print(score)

-2


In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
# Gilbert, C. H. E. (2014). Vader: A parsimonious rule-based model for sentiment analysis of social media text. In Eighth International Conference on Weblogs and Social Media (ICWSM-14). Available at (20/04/16) http://comp. social. gatech. edu/papers/icwsm14. vader. hutto. pdf.
from nltk import tokenize
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
sid = SentimentIntensityAnalyzer()

def get_senti_score_nltk(text):
    text_sents = tokenize.sent_tokenize(text)
    sum_desc_com = 0

    for sentence in text_sents:
        ss = sid.polarity_scores(sentence)
        compound = ss['compound']
        sum_desc_com += compound

    try :
        avg_text_com = sum_desc_com/len(text_sents)
    except:
        avg_text_com = 'NA'

    return avg_text_com

In [ ]:
avg_sent_score = get_senti_score_nltk(example1)

print('avg score : ', avg_sent_score)

avg score :  0.5106


In [ ]:
text2 = 'I watched a newly released movie yesterday. It was an action movie. It was great.'
text3 = "We love the speaker which is why this review is disappointing me for. The item description talks about the Bluetooth technology inside but Sonos is a very closed system. You cannot play music you own unless you buy the $400 Sonos connect in addition to these speakers. We have a bluetooth turntable that is not Sonos brand that connects to all our other Bluetooth speakers fine and we cannot use it with our Sonos. You can only play music from the programs inside the Sonos app so only streaming services. If you have music you purchased through iTunes, cds you’ve saved to your computer, or a Bluetooth record player like us your tough out of luck unless you’re willing to drop a couple hundred bucks more. It feels very deceiving and disappointing that you can only use what they approve. "
text4 = " I bought this pair originally to serve as rear surroundsound speakers to my existing home theater system, which I bought the Sonos Connect for ( rather than paying more for wireless front speakers and subwoofer). While they worked fantastic for this application, I quickly realized that the quality of sound, even at high volume levels, meant that I was under estimating their ability. I transitioned one further back so it could double as a functional speaker in my kitchen, and the other is in my dining room where it still provides surroundsound when needed for my home theater system but adds extra music ability for that room.I've got to say that I seriously underestimated the sound quality of these as I thought that the transmission through wireless would leave them somewhat lacking. I'm very happy to say I was wrong and that these are fantastic speakers. I'm going to buy several more for my master bedroom and bath, and workout room."

avg_sent_score = get_senti_score_nltk(text3)
print('avg score: ', avg_sent_score)

avg score:  0.07044285714285713
